In [ ]:
cd ../..

In [ ]:
import os
import numpy as np
import pandas as pd
from numpy.random import randn
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, Conv1DTranspose, MaxPool1D, GlobalMaxPool1D, Flatten, Dropout, LeakyReLU, Reshape, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
from gans.utils import create_gan, train_gan
from utils.windowing import windowing_dataframe, transform_windows_df
from utils.preprocess import filter_by_activity_index, calc_consultant
from utils.utils import check_dir_creation
from utils.plotting import plot_n_heatmaps, plot_n_lineplots
import seaborn as sns

In [ ]:
def create_discriminator(input_shape, optimizer=Adam(learning_rate=0.0001, clipvalue=1.0, decay=1e-8)):
    discriminator = Sequential()
    discriminator.add(Conv1D(8, kernel_size=9, activation='relu', input_shape=input_shape))
    discriminator.add(Flatten())
    discriminator.add(Dense(1, activation='sigmoid'))

    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return discriminator

In [ ]:
def create_generator(latent_dim, kernel_size, kernel_num=3, small=True):
    if small:
        generator = Sequential()
        generator.add(Dense(125 * 3, kernel_initializer='he_uniform', input_shape=(latent_dim,)))
        generator.add(LeakyReLU())
        generator.add(BatchNormalization())

        generator.add(Reshape((125, 3)))

        generator.add(Conv1D(kernel_num, kernel_size=kernel_size, kernel_initializer='he_uniform', padding='same'))
        generator.add(LeakyReLU())
        generator.add(BatchNormalization())

        generator.add(Conv1DTranspose(kernel_num, kernel_size=4, strides=2, padding='same'))
        generator.add(LeakyReLU())
        generator.add(BatchNormalization())

        generator.add(Conv1D(3, kernel_size=kernel_size, padding='same', kernel_initializer='he_uniform', activation='linear'))
    else:
        generator = Sequential()
        generator.add(Dense(125 * 3, kernel_initializer='he_uniform', input_shape=(latent_dim,)))
        generator.add(LeakyReLU())
        generator.add(BatchNormalization())
        
        generator.add(Reshape((125, 3)))

        generator.add(Conv1D(kernel_num, kernel_size=kernel_size, kernel_initializer='he_uniform', padding='same'))
        generator.add(LeakyReLU())
        generator.add(BatchNormalization())

        generator.add(Conv1D(kernel_num, kernel_size=kernel_size, kernel_initializer='he_uniform', padding='same'))#
        generator.add(LeakyReLU())
        generator.add(BatchNormalization())

        generator.add(Conv1DTranspose(kernel_num, kernel_size=4, strides=2, padding='same'))
        generator.add(LeakyReLU())
        generator.add(BatchNormalization())

        generator.add(Conv1D(kernel_num, kernel_size=kernel_size, kernel_initializer='he_uniform', padding='same'))
        generator.add(LeakyReLU())
        generator.add(BatchNormalization())

        generator.add(Conv1D(kernel_num, kernel_size=kernel_size, kernel_initializer='he_uniform', padding='same'))
        generator.add(LeakyReLU())
        generator.add(BatchNormalization())

        generator.add(Conv1D(kernel_num, kernel_size=kernel_size, padding='same', kernel_initializer='he_uniform', activation='linear'))
        
    #generator.summary()
    return generator

In [ ]:
act_id = 0

latent_dim = 64
kernel_num=3
kernel_size=5
small=True

steps = 1000
batch_size = 32

num_gen = 5000
add_sample_num = 1000

In [ ]:
train_df = pd.read_hdf('./datasets/mydata/train_df.h5')
val_df = pd.read_hdf('./datasets/mydata/val_df.h5')
test_df = pd.read_hdf('./datasets/mydata/test_df.h5')

In [ ]:
window_size = 5*50
step_size = int(window_size/2)
col_names = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z', 'userAcceleration.c']
method ='sliding'

In [ ]:
train_windowed_df = windowing_dataframe(train_df, window_size=window_size, step_or_sample_size=step_size, col_names=col_names, method=method)
val_windowed_df = windowing_dataframe(val_df, window_size=window_size, step_or_sample_size=step_size, col_names=col_names, method=method)
test_windowed_df = windowing_dataframe(test_df, window_size=window_size, step_or_sample_size=step_size, col_names=col_names, method=method)

In [ ]:
input_cols = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z']
x_train, y_train = transform_windows_df(train_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)
x_val, y_val = transform_windows_df(val_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)
x_test, y_test = transform_windows_df(test_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)

In [ ]:
x_train_activity, _ = filter_by_activity_index(x=x_train, y=y_train, activity_idx=act_id)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
labels=["standing", "walking", "jogging"]

In [ ]:
input_cols = ['userAcceleration.c']
x_train_clf, y_train_clf = transform_windows_df(train_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)
x_val_clf, y_val_clf = transform_windows_df(val_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)
x_test_clf, y_test_clf = transform_windows_df(test_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)

In [ ]:
x_train_clf = x_train_clf.reshape((len(x_train_clf), window_size))
x_val_clf = x_val_clf.reshape((len(x_val_clf), window_size))
x_test_clf = x_test_clf.reshape((len(x_test_clf), window_size))

In [ ]:
svm_clf = SVC()
svm_clf.fit(x_train_clf, y_train_clf)

#### train performance

In [ ]:
y_train_head = svm_clf.predict(x_train_clf)
print(classification_report(y_train_clf, y_train_head, target_names=labels))

In [ ]:
cm = confusion_matrix(y_train_clf, y_train_head)
cm_df = pd.DataFrame(cm, index = labels,
                  columns = labels)
sns.heatmap(cm_df, annot=True, cmap='YlGnBu', fmt='g')

In [ ]:
orig_train_acc = accuracy_score(y_train_clf, y_train_head)
orig_train_acc

#### test performance

In [ ]:
y_test_head = svm_clf.predict(x_test_clf)

In [ ]:
print(classification_report(y_test_clf, y_test_head, target_names=labels))

In [ ]:
cm = confusion_matrix(y_test, y_test_head)
cm_df = pd.DataFrame(cm, index = labels,
                  columns = labels)
sns.heatmap(cm_df, annot=True, cmap='YlGnBu', fmt='g')

In [ ]:
orig_test_acc = accuracy_score(y_test_clf, y_test_head)
orig_test_acc

In [ ]:
### GAN

In [ ]:
input_shape = x_train[0].shape
discriminator = create_discriminator(input_shape);
generator = create_generator(latent_dim, kernel_num=kernel_num, kernel_size=kernel_size, small=small)
gan = create_gan(generator_model=generator, discriminator_model=discriminator)

In [ ]:
train_gan(generator, discriminator, gan, x_train_activity, steps)

In [ ]:
## eval

In [ ]:
random_latent_vectors = np.random.normal(size=(num_gen, latent_dim))
generated_sensor_data = generator.predict(random_latent_vectors)

In [ ]:
gen_df = pd.DataFrame(np.array([ts.transpose() for ts in generated_sensor_data]).tolist(), columns= ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z'])
gen_df['userAcceleration.c'] = calc_consultant(gen_df)
gen_df['act'] = act_id

In [ ]:
gen_windowed_df = windowing_dataframe(gen_df, window_size=window_size, step_or_sample_size=step_size, col_names=col_names, method=method)
    
input_cols = ['userAcceleration.c']
x_gen, y_gen = transform_windows_df(gen_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)
x_gen = x_gen.reshape((len(x_gen), window_size))

x_train_gen = np.concatenate([x_train_clf, x_gen[:add_sample_num]])
y_train_gen = np.concatenate([y_train_clf, y_gen[:add_sample_num]])

In [ ]:
svm_clf = SVC()
svm_clf.fit(x_train_gen, y_train_gen)
    
y_train_head = svm_clf.predict(x_train_clf)
train_acc = accuracy_score(y_train_clf, y_train_head)
    
y_test_head = svm_clf.predict(x_test_clf)
test_acc = accuracy_score(y_test_clf, y_test_head)

In [ ]:
print('Gen Train acc:', train_acc, 'vs.', orig_train_acc, ':Orig Train Acc')
print('Gen Test acc:', test_acc, 'vs.', orig_test_acc, ':Orig Test Acc')

In [ ]:
plot_n_heatmaps(x_train_activity, generated_sensor_data, n=10)

In [ ]:
plot_n_lineplots(x_train_activity, generated_sensor_data, n=10)

In [ ]:
for small in [True, False]:
    for act_id in range(0,3):
        for latent_dim in [32, 64, 128, 256]:
            for kernel_size in [3, 5, 7, 9, 11, 13, 15]:
                x_train_activity, _ = filter_by_activity_index(x=x_train, y=y_train, activity_idx=act_id)
                
                input_shape = x_train[0].shape
                discriminator = create_discriminator(input_shape);
                generator = create_generator(latent_dim, kernel_num=kernel_num, kernel_size=kernel_size, small=small)
                gan = create_gan(generator_model=generator, discriminator_model=discriminator)

                train_gan(generator, discriminator, gan, x_train_activity, steps)
                random_latent_vectors = np.random.normal(size=(num_gen, latent_dim))
                generated_sensor_data = generator.predict(random_latent_vectors)

                gen_df = pd.DataFrame(np.array([ts.transpose() for ts in generated_sensor_data]).tolist(), columns= ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z'])
                gen_df['userAcceleration.c'] = calc_consultant(gen_df)
                gen_df['act'] = act_id

                gen_windowed_df = windowing_dataframe(gen_df, window_size=window_size, step_or_sample_size=step_size, col_names=col_names, method=method)

                input_cols = ['userAcceleration.c']
                x_gen, y_gen = transform_windows_df(gen_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)
                x_gen = x_gen.reshape((len(x_gen), window_size))

                x_train_gen = np.concatenate([x_train_clf, x_gen[:add_sample_num]])
                y_train_gen = np.concatenate([y_train_clf, y_gen[:add_sample_num]])

                svm_clf = SVC()
                svm_clf.fit(x_train_gen, y_train_gen)

                y_train_head = svm_clf.predict(x_train_clf)
                train_acc = accuracy_score(y_train_clf, y_train_head)

                y_test_head = svm_clf.predict(x_test_clf)
                test_acc = accuracy_score(y_test_clf, y_test_head)

                plot_n_heatmaps(x_train_activity, generated_sensor_data, n=10, save_dir='out/cnn_steps-{}_latentdim-{}_kernelnum-{}_kernelsize-{}_small-{}_actId-{}'.format(steps, latent_dim, kernel_num, kernel_size, small, act_id))
                plot_n_lineplots(x_train_activity, generated_sensor_data, n=10, save_dir='out/cnn_steps-{}_latentdim-{}_kernelnum-{}_kernelsize-{}_small-{}_actId-{}'.format(steps, latent_dim, kernel_num, kernel_size, small, act_id))

                file = open('out/cnn_steps-{}_latentdim-{}_kernelnum-{}_kernelsize-{}_small-{}_actId-{}/acc.txt'.format(steps, latent_dim, kernel_num, kernel_size, small, act_id), 'w+')
                file.write(f'Gen Train acc: {train_acc} vs. {orig_train_acc} :Orig Train Acc \n')
                file.write(f'Gen Test acc: {test_acc} vs. {orig_test_acc} :Orig Test Acc')
                file.close()

In [ ]:
#labels = np.concatenate([smooth_labels(np.zeros((batch_size, 1)), 0.0, 0.3), smooth_labels(np.ones((batch_size, 1)), -0.3, 0.3)])
#labels = np.concatenate([np.zeros((batch_size, 1)), np.ones((batch_size, 1))])

def smooth_labels(labels, lower, upper, round_decimel=1, random_state=None):
    n_samples = len(labels)
    if random_state:
        np.random.seed(random_state)
    return labels + np.round(np.random.uniform(lower, upper, n_samples), round_decimel)[:, None]